# Расчет предельных режимов работы двигателя

In [ ]:
import numpy as np
from pmsm import ModelWithNominalModeRotary

# APM-FEP30AMK3 as default model
m = ModelWithNominalModeRotary('APM-FEP30AMK3', n_pole_pairs = 4, 
                         rated_ac_voltage = 220, rated_power = 3000, rated_current = 9.94, rated_speed = 3000 * 2*np.pi/60, 
                         Rll = 1.3, Lll = 0.0047,
                         J = 19.04e-4)

### Задание модели

Полная модель может быть рассчитана по парметрам номинального режима и значениям индуктивности и сопротивления.

* Обмотки подключены звездой.
* Линейная индуктивность `Lll` может быть напрямую измерена мультиметром между выходными терминалами даигателя.
* Для Невнополюсных машин можно ввести `Ld` и `Lq`.
* Линейное соротивление `Rll` модет быть измерено так же.
* Номинальные параметры должны быть приведены в документации двигателя. Обратите внимания, что обычно они 1) rms, 2) напряжение может быть линеным или фазным.

После нажатия `Calculate`, если введенные данные не противречивы, долна быть рассчитана модель двигателя.

In [ ]:
from forms import AttributeForm, cprint

model_form = AttributeForm(ModelWithNominalModeRotary, ['poles','resistanse', 'inductance', 'inductance', 'inertia', 'rated_torque', 'rated_speed', 'rated_voltage', 'rated_current', 'rated_power'], m)

### Диаграмма скорость-момент

На основе парметров двигателя рассчитывается диграмма для различных схем управления.
* **MTPV** (Max Torque per Volt)
  Характеризует ограничение на момент со стороны напряжения (зона пикового момента). В большинстве ситуаций номинальный ток значительно ниже.
* **MTPA** (Max Trorque per Ampere)
  Базовая схема для FOC (прямой ток $I_d = 0$) при его идеальной реализации при указанных ограничениях тока и напряжения. Расчет немного упрощен, не учитывает явнополюсность (у явнополюсных машин можно достигнуть немного большего момента при $I_d \ne 0$).
* **FW** (Flux Weakining)
  Режим высоких скоростей при учете  ограничений тока и напряжения (прямой ток $I_d < 0$). Расчитан упрощенно без учета сопротивления и явнополюсности. У нас не планируется к реализации.
* **MTPA AB-frame** (Max Trorque per Ampere в неподвижной СК)
  Вариант MTPA при реализации его в неподвижной СК (наша реализация). Максимальный момент зависит от быстроодействия регулятора тока (Current Treg).

  

In [ ]:
import matplotlib.pyplot as plt
from foc_base import PointDQ
from pmsm import ModelRotary
from collections import namedtuple 

PIParameters = namedtuple('PIParameters', 'kp ki')

def MTPA_in_ABframe(m, v, Im, Um, ireg):
    jw = complex(0, v*m.N)
    Wjw = ((ireg.kp - v*m.N*m.Fm/Im)*jw + ireg.ki) / (m.L*jw**2 + (m.R+ireg.kp)*jw + ireg.ki)
    Idq = complex(0, Im) * Wjw
    Id, Iq = np.real(Idq), np.imag(Idq)
    # calculate voltage
    Ud = m.R*Id - v*m.N*m.Lq*Iq
    Uq = m.R*Iq + v*m.N*m.Ld*Id + v*m.N*m.Fm  
    T = 3/2*m.N*m.Fm*Iq
    # check limits
    if Ud**2 + Uq**2 > Um**2:
        T = np.nan
    return T, PointDQ(Id, Iq), PointDQ(Ud, Uq)

def rpm_to_rads(rpm):
    return np.pi/30*rpm
def rads_to_rpm(rads):
    return rads*30/np.pi

def MTPA(m, v, Im, Um):
    # parameters
    a = (v*m.N*m.Fm)**2
    b2 = 2 * v*m.N*m.Fm * m.R
    c2 = m.R**2 + (v*m.N*m.Lq)**2
    # check if current can be maxinal
    if a + b2*Im + c2*Im**2 <= Um**2:
        Iq = Im
    else:
        # calculate maximal Iq
        # quadric eq: (a - Um**2) + b2*Iq + c2*Iq^2 = 0
        D = b2**2 - 4*(a - Um**2)*c2
        if D >= 0:
            Iq = (-b2 + np.sqrt(D))/(2*c2)
        else:
            Iq = np.nan
    # dependent parameters
    T = 3/2*m.N*m.Fm*Iq
    Ud = - v*m.N*m.Lq*Iq
    Uq = m.R*Iq + v*m.N*m.Fm
    return T, PointDQ(0.0, Iq), PointDQ(Ud, Uq)

def MTPV(m , v, Im, Um):
    a = (v*m.N*m.Fm)**2
    b1 = 2 * v*m.N*m.Fm * v*m.N*m.Ld
    b2 = 2 * v*m.N*m.Fm * m.R
    c1 = m.R**2 + (v*m.N*m.Ld)**2
    c2 = m.R**2 + (v*m.N*m.Lq)**2
    d = 2 * v*m.N*(m.Ld - m.Lq) * m.R
    # solve quadratiq eq
    aa = a - Um**2 - b1**2/(4*c1)
    bb = b2 - b1*d/(2*c1)
    cc = c2 - d**2/(4*c1)
    D = bb**2 - 4*aa*cc
    if D >= 0:
        Iq = (-bb + np.sqrt(D))/(2*cc)
    else:
        Iq = np.nan
    Id = - (b1 + d*Iq)/(2*c1)
     # check current limits
    if Im is not None and Id**2 + Iq**2 > Im**2:
        Id, Iq = np.nan, np.nan
    # dependent parameters
    T = 3/2*m.N*(m.Fm*Iq + (m.Ld - m.Lq)*Iq*Id)
    Ud = m.R*Id - v*m.N*m.Lq*Iq
    Uq = m.R*Iq + v*m.N*m.Ld*Id + v*m.N*m.Fm  
    return T, PointDQ(Id, Iq), PointDQ(Ud, Uq)

from typing import NamedTuple, Type, Dict, Sequence, List, Any, Callable
import ipywidgets as widgets
from IPython.display import  display, HTML
import matplotlib.pyplot as plt

from forms import ParameterWidgets, ParameterForm

class ModeForm(ParameterForm):                    
    def __init__(self, model: ModelRotary):
        self._model = model
        # current and voltage
        self._current = ParameterWidgets('Max current', round(model.In, 2), round(model.In, 2), model.In.units, 'Current limit')
        self._voltage = ParameterWidgets('Max voltage', round(model.Un, 2), round(model.In, 2), model.Un.units, 'DC voltage limit')
        self._current_treg = ParameterWidgets('Current Treg', 0.001, 1.0, 's', 'Current regulator settling time')
        # call superclass constructor
        super(ModeForm, self).__init__('Torque-Speed plot', [ self._current, self._voltage, self._current_treg ])

    def calculate(self):
        m = self._model
        Um = self._voltage.value
        Im = self._current.value
        t_Ireg = self._current_treg.value

        #plot parameters
        n_points = 1000
        vlim_rpm = np.array([1, 3*rads_to_rpm(m.vn)])
        Tlim = np.array([0, m.Tn*5])
        vlim = rpm_to_rads(vlim_rpm)
        v = np.linspace(vlim[0], vlim[1], n_points)

        ### plot modes ###
        fig, ax = plt.subplots()
        # nominal mode
        ax.plot(rads_to_rpm(m.vn), m.Tn, 'o', label='nominal', linewidth=3)

        # MTPV
        # plot unconstarined curve
        T = np.vectorize(lambda v: MTPV(m, v, Im = None, Um = Um)[0])(v) 
        ax.plot(rads_to_rpm(v), T, label='MTPV (no current limit)')
        # plot constarined curve
        T = np.vectorize(lambda v: MTPV(m, v, Im = Im, Um = Um)[0])(v) 
        ax.plot(rads_to_rpm(v), T, label='MTPV (current limited)', linewidth=3)

        # MTPA
        T = np.vectorize(lambda v: MTPA(m, v, Im = Im, Um = Um)[0])(v) 
        ax.plot(rads_to_rpm(v), T, label='MTPA', linewidth=3)

        # FW
        Iq = np.full(v.shape, np.nan)
        v_mtpa0 = np.sqrt( Um**2 / ((m.N*m.L)**2 * ((m.Fm/m.L)**2 + Im**2)) )
        v_mtpa1 = np.sqrt( Um**2 / ((m.N*m.L)**2 * (m.Fm/m.L)**2 ) )
        cosA = ( (Um/(m.N*m.L*v))**2 - Im**2 - (m.Fm/m.L)**2 ) / ( 2 * Im * m.Fm/m.L )
        inds = (cosA >= -1) & (cosA <= 1) & (v >= v_mtpa0)
        Iq[inds] = Im*np.sqrt(1 - cosA[inds]**2)
        T = 3/2*m.N*m.Fm*Iq
        # plot FW
        ax.plot(rads_to_rpm(v), T, label='FW', linestyle='--', linewidth=3)

        # MTPA in AB fraame
        # current regulator    
        k = 3/t_Ireg
        ireg = PIParameters(m.L*k, m.R*k)
        # plot MTPA
        T = np.vectorize(lambda v: MTPA_in_ABframe(m, v, Im = Im, Um = Um, ireg = ireg)[0])(v) 
        ax.plot(rads_to_rpm(v), T, label='MTPA ab-frame', linewidth=3)

        # plot properties
        ax.set_xlim(*rads_to_rpm(vlim))
        ax.set_ylim(*Tlim)
        ax.set_xlabel('v, rpm')
        ax.set_ylabel('T, Hm')
        ax.set_title(m.name)
        ax.grid(True)
        ax.legend()

        plt.show()

form = ModeForm(model_form.model)

### Подбор парметров профиля скорости

При заданных лимитах (напряжение, сила тока) и  парметрах траектории отображаются парметры траектории.
* Лимиты по скорости и ускорению значитмы: при их превышении наверняка двигатель выйдет за номинальный режим.
* Лимиты по рывку рассчитан из худшей ситуации (при максимальной скорости и ускорении надо дать максимальный рывок). Есть вариант более точного расчета, учитывающего параметры профиля, но разница между ними небольшая.
* Все графики построены для идеальнго MTPA (dq-frame) с нулевым сопротивленим обмоток. Из-за последнего лимиты будут достигатяс чуть раньше (величина порядка $RI_{max}$ по напряжению).

<details>
<summary><b>Расчет ограничений...</b></summary>

### Ограничения режимов СДПМ
 
В статическом режиме поведение определяется соотношениями 
$$
RI_d - \omega L I_q = U_d,
$$
$$
RI_q + \omega L I_d + \omega \lambda_m  = U_q,
$$
где
* $R$ --- сопротивление, $L$ --- индуктивность, $\lambda_m$ --- потокосцепление ротора,
* $N$ --- число пар полюсов на единицу смещения,
* $I_d$, $I_q$ --- токи, $U_d$, $U_q$ --- напряжения,
* $\omega$ --- электрическая скорость.

Реальная скорость выхдного вала будет
$v = \omega / N$.
  

#### Ограничение амплитуды напряжения
Это ограничение не может быть превышено без потери управления и в общем случае представляет элипс на плоскости $(I_d, I_q)$:
$$
(RI_d - \omega L I_q)^2 + (RI_q + \omega L I_d + \omega \lambda_m)^2 < U_{max}^2.
$$
В зависимсоти от скорости и типа двигателя могут получатсья разные формы:
* Выссокоомная обмотка, малая скорсоть: окружность фиксированного радиуса с центром в точке $(0,0)$: $I_d^2 + I_q^2 < (U_{max}/R)^2$.
* Низкоомные обмотки: окружность с центром в точке $(0,-\lambda/L)$ и радиусом, лавиясщим от скорости: $(I_d - \lambda_m\omega)^2 + I_q^2 < (U_{max}/(L\omega))^2$.
**Далее везде речь идет о двигателях с низкоомными обмотками**.
  
#### Ограничение амплитуды тока 
Это не абсолютное оганичение, продолжительный или пиковый ток могут быть привешены, взможно, с разрушительными последствиями для двигателя.
$$
I_d^2 + I_q^2 < I_{max}^2.
$$

#### Влияние режима работы 
Момент/сила создаваемая двигателем определяется в первую очередь квадартурным током: 
$$
F = 3/2N\lambda_m I_q.
$$
В случае если машина неявнополюсная, добавляется еще слагаемое $3/2N(L_d - Lq)I_d I_q$.

Выбор $I_q$ свободен, что дает некторую свободу в выборе режима.
* **MTPA**: $I_d = 0$ --- "максимальный момент на ампер" для явнополюсных машин. 
* **FW**: $I_d < 0$ --- понижение потокосцепление, позволяет достичь больших скорсотей, чем MTPA
* есть ряд других схем.

**Далее везде в тексте предполагается режим MTPA**, т.к. только он у нас реализован. Однако эти расчеты не будут верны, если происходит что-то типа такого https://ximc.ru/issues/118210, поскольку СУ не может выдержать $I_d > 0$ максимально достижимый $I_q$ становится меньше.

## Максимальная скорость

Из уравнения 
$$
RI_q + vN L I_d + vN \lambda_m  = U_q,
$$
в преполажениях **корректно работающей стратегии MTPA**, **низкоомных обмоток** и **малой нагрузки** основное ограничение имеет вид
$$
|\omega| \le \frac{U_q}{\lambda_m},
$$
что при пересчете в скорость $v$ даст формулу 
$$
|v| \le \frac{U_q}{N\lambda_m}
$$

## Максимальнре ускорение

При условии корректной работы регулятора и лимитера тока **максимальный момент/сила** ограничены следующим выражением
$$
F_{max}(\omega) \le 3/2N\lambda_m \max \left\{I_{limit}, \frac{1}{L}\sqrt{\left(\frac{U_{max}}{\omega}\right)^2 - \lambda_m^2} \right\}, 
$$
где 
* $\omega = N v$ --- электрическая скорость,
* $\lambda_m$ --- потокосцепение,
* $N$ --- число пар полюсов для ротационного двигателя или $N =  \frac{\pi}{\tau}$ для линеного,
* $\tau$ --- расстояние между полюсами,
* $L$ --- эффективная индуктивность, но это *не та индуктивность, что дается в документации*, обычно фазовая или линейная.
Параметры двигателя можно определить путем идентификации или по datasheet: https://ximc.ru/projects/ximcresearch/wiki/%D0%9E%D0%B1%D1%89%D0%B0%D1%8F_%D0%B8%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D1%8F_%D0%BE_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8F%D1%85_%D0%B8_%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D0%B0%D1%85_%D0%A8%D0%94_%D0%B8_%D0%A1%D0%94%D0%9F%D0%9C_%D0%98%D0%BD%D1%82%D0%B5%D1%80%D0%BF%D0%B5%D1%80%D1%82%D0%B0%D1%86%D0%B8%D1%8F_datasheet

Тогда ускорение ограничено выраженим
$$
a_{max} = \frac{3 N\lambda_m}{2 J} \max \left\{I_{limit}, \frac{1}{L}\sqrt{\left(\frac{U_{max}}{\omega}\right)^2 - \lambda_m^2} \right\},
$$
где $J$ --- масса или момент.

Уравнение не учитывает трение, его эффект может быть легко добавлен:
$$
a_{max} = \frac{1}{J}\max\{F_{max}(\omega) - F_q - \nu v, 0\},
$$
где $F_q$ --- трение Кулона, $\nu$ --- вязкое трение,  $v$ --- скорость. Отмечу, что **обычно эффект трения незначителен.**

**Таким образом максимальное ускорение --- функция скорости: $a_{max}(v)$.**

## Максимальный рывок

Максимальную скорсоть роста тока (а значит и момента/силы) можно получить из уравнения 
$$
L\dot{I}_q + RI_q = U_q - \omega \lambda_m.
$$
Здесь опять преполагается **использование стратегии MTPA**, т.е.
$$
RI_d - \omega L I_q = U_d.
$$

Результирующее ограничение 
$$
U_{qmax}(\omega, I_q) = \sqrt{U_{max}^2 - \omega^2 L^2 I_q},
$$
$$
\frac{3 N \lambda_m}{2 m L} ( -U_{qmax}(\omega, I_q) - \omega \lambda_m ) \le j \le \frac{3 N \lambda_m}{2 m L} ( U_{qmax}(\omega, I_q) - \omega \lambda_m ).
$$

**Таким образом максимальный рывок --- функция скорости и ускорения: $j_{max}(v, a)$.**

### Максимальной практическое ограничение рывка (уровень 1)

Максмальное теоретическое ускорение не будет наблюдаться на практике, если для регулирования тока не используется какой-нибудь релейный закон управления. На практике же **рывок будет ограничиваться быстродействием регулятора тока**.

За основу можно приянть следующую формулу:
$$
\omega_{Icp} I_{qmax} \approx \dot{I}_{qmax} \text{ или } \omega_{Icp} a_{max} \approx j_{max},
$$
где $\omega_{Icp}$ --- частота среза регулятора тока.

Это соотношение весьма условно, так на частоте среза гармонический сигнал искажен на 70% по амплитуде. Если нужно долше точности, то надо более шировкая полоса пропускания с более высокой частотой среза: 


| Частота среза | Искажения амплитуды на гармоническом сигнале | 
|:--------------:|:-----------------------:|
| $\omega_{Icp} > \frac{j_{max}}{a_{max}}$ | 30% |
| $\omega_{Icp} > 3\frac{j_{max}}{a_{max}}$ | 5% |
| $\omega_{Icp} > 5\frac{j_{max}}{a_{max}}$ | 2% |
| $\omega_{Icp} > 10\frac{j_{max}}{a_{max}}$ | 0.5% |

**Т.е. частота $\frac{j_{max}}{F_{max}}$ должна попадать в полосу пропускания контура тока.**

### Максимальной практическое ограничение рывка (уровень 2)

Ограничение выше будет достигатсья только в том случае, если **используется упреждающее управление по силе**. В нашей системе это возможно только с нашим драйвером или внешним дравером в режиме тока, оно долно учитывать массу нагрузки и параметры двигателя.

Иначе 
$$
\omega_{vcp} F_{max} = a_{max},
$$
где $\omega_{vcp}$ --- частота среза регулятора скорости, котрая обычно меньше частоты среза контура тока.

**Т.е. частота $\frac{a_{max}}{F_{max}}$ должна попадать в полосу пропускания контура скорости.**
  
</details>


In [ ]:
import ipywidgets as widgets
from IPython.display import  display, HTML
import matplotlib.pyplot as plt

from forms import ParameterWidgets, ParameterForm
import ruckig

def accel_limit(m, v, Um):
    s = (Um/(m.N*v))**2 - m.Fm**2
    if s >= 0:
        Iqmax = min(m.In, np.sqrt(s)/m.L)
    else:
        Iqmax = 0.0
    Tmax = 3/2*m.N*m.Fm*Iqmax
    amax = Tmax / m.J
    return amax

def jerk_limit(m, v, a, Um):
    Iq = (m.J * a) / (3/2*m.N*m.Fm)
    d = Um**2 - (v*m.N*m.L*Iq)**2
    Uqmax = np.sqrt(d) if d >= 0.0 else 0.0
    dIqmax = max(Uqmax - v*m.N*m.Fm,0)/m.L
    jmax = 3/2*m.N*m.Fm*dIqmax / m.J
    return jmax

def jerk_limit_profile(m, vmax, amax, Um, debug = False, state = False):
    # jerk limit for given speed under assumption of "the worst case scenario" on given profile
    # 1) traget speed is vm, max acceleration is am
    # 2) current speed is v, accel a is just reached applying jerk jmax,
    # 3) from now on accel decreases with jmax rate to zero until traget speed is reached

    # calulate jmax and initial accel for given initial speed
    K = (3*m.N*m.Fm)/(2*m.J*m.L) 
    def jmax_limit(v):
        # deceleration contraint: a^2 = 2*jmax*(vm - v)
        # jmax limit jest before deceleration: 
        # jmax <= (3/2*N*Fm/J) * 1/L * (sqrt(Um^2 - (v*N*L*Iq)^2) - v*N*Fm), 
        # Iq = a / (3/2*N*Fm/J)
        # solve quadric equation
        bb1 = K * m.N*m.Fm*v + m.N**2 * (vmax - v) * v**2
        cc = K**2 * ((m.N*m.Fm*v)**2 - Um**2)
        D1 = bb1**2 - cc
        jmax = - bb1 + np.sqrt(D1)
        # calulate accel
        a = np.sqrt(2*jmax*(vmax - v))
        return jmax, a
        
    # Enforce accel limit. It is cleary satisfied at v = vm
    # binary search of left limit
    vmin = vmax / 2
    vstep = vmax / 4
    for k in range(16):
        jmax, a = jmax_limit(vmin)
        if a > amax:
            vmin += vstep
        else:
            vmin -= vstep
        vstep *= 0.5
        
    # create grid and find max min jerk value
    v = np.linspace(vmin, vmax, 1000)
    jmax, a = jmax_limit(v)
    # debug output
    if debug:
        k = np.argmin(jmax)
        # check if solution is correct
        Iq = (m.J*a[k]) / (3/2*m.N*m.Fm)
        d = Um**2 - (v[k]*m.N*m.L*Iq)**2
        Uqmax = np.sqrt(d) if d >= 0.0 else 0.0
        dIqmax = max(Uqmax - v[k]*m.N*m.Fm,0)/m.L
        jmax_c = 3/2*m.N*m.Fm*dIqmax / m.J
        print(f'v = {v[k]} rad/s, a = {a[k]} rad/s^2, Iq = {Iq} A, Uqmax = {Uqmax} V, L*dIqmax = {m.L*dIqmax} A/s, jmax = {jmax_c} rad/s^3')
        # plot
        fig, (ax1, ax2) = plt.subplots(2,1)
        ax1.plot(v, jmax)
        ax1.set_xlabel('v, rad/s')
        ax1.set_ylabel('j, rad/s^3')
        ax1.grid(True)
        ax2.plot(v, a)
        ax2.plot(v, am*np.ones(v.shape))
        ax2.set_xlabel('v, rad/s')
        ax2.set_ylabel('a, rad/s^2')
        ax2.grid(True)
        plt.show()
    # retyrn worst-case scehario
    if not state:
        return np.min(jmax)
    else:
        k = np.argmin(jmax)
        return jmax[k], v[k], a[k]

class ProfileForm(ParameterForm):                    
    def __init__(self, model: ModelWithNominalModeRotary):
        self._model = model
        # current and voltage
        self._current = ParameterWidgets('Max current', round(model.In, 2), round(model.In, 2), model.In.units, 'Current limit')
        self._voltage = ParameterWidgets('Max voltage', round(model.Un, 2), round(model.In, 2), model.Un.units, 'DC voltage limit')
        # profile properties 
        T = 0.2
        vn = model.vn
        self._distance = ParameterWidgets('Profile distance', round(vn*2.1*T, 2), +np.inf, 'rad', 'Distance of movement.')
        self._speed = ParameterWidgets('Profile speed', round(vn, 2), 0.0, 'rad/s', 'Maximal speed.')
        self._accel = ParameterWidgets('Profile accel', round(vn/T, 2), 0.0, 'rad/s^2', 'Maximal acceleration.')
        self._jerk = ParameterWidgets('Profile jerk', round(vn/T**2, 2), 0.0, 'rad/s^3', 'Maximal jerk.')
        # trajctory selector
        self._worst_case_checkbox = widgets.Checkbox(value=False, description='Worst case trajectory (maximal velocity, accel, jerk is reached)')
        # call superclass constructor
        super(ProfileForm, self).__init__('Speed profile configurator', 
                                          [self._current, self._voltage, self._distance, self._speed, self._accel, self._jerk],
                                          custom_widget = self._worst_case_checkbox)

    def limit_params(self):
        return self._current.value, self._voltage.value
    
    def profile_params(self):
        return self._distance.value, self._speed.value, self._accel.value, self._jerk.value

    def update(self):
        m = self._model
        Im, Um = self.limit_params()
        _, v, a, _ = self.profile_params()
        
        # speed limit (simpified MTPA)
        vmax = Um / (m.Fm*m.N)
        # acceleration limit
        amax = accel_limit(m, v, Um)
        # jerk limit
        #jmax = jerk_limit(m, v, a, Um)
        jmax = jerk_limit_profile(m, v, a, Um)

        # assign limits
        self._speed.max_value = round(vmax, 2)
        self._accel.max_value = round(amax, 2)
        self._jerk.max_value = round(jmax, 2)

    def calculate(self):
        T = 1/20000
        s, vmax, amax, jmax = self.profile_params()
        Im, Um = self.limit_params()
        
        # calculate critical times
        T3 = amax/jmax
        T2 = vmax/amax
        T1 = s/vmax

        # display related information
        display(HTML('<h3>Trajectory time parameters</h3>'))
        display(HTML('$T_1 = \cfrac{S}{v_{max}} = $ %.4f s, $T_2 = \cfrac{v_{max}}{a_{max}} =$ %.4f s, $T_3 = \cfrac{a_{max}}{j_{max}} =$ %.4f s' % (T1, T2, T3)))
        # check amax is reaced
        if T2 >= T3:
            cprint('Max acceleration is reached: $T_2 \ge T_3$.', color = 'green')
        else:
            cprint('Max acceleration is NOT reached: $T_2 < T_3$.', color = 'red')
            print('    Increase: jerk or velocity.')
            print('    Decrease: acceleration.')
        # check vmax is reaced
        if T1 >= (T2 + T3):
            cprint('Max velocity is reached: $T_1 \ge (T_2 + T_3)$.', color = 'green')
        else:
            cprint('Max velocity is NOT reached: $T_1 < (T_2 + T_3)$.', color = 'red')
            print('    Increase: distance, acceleration or jerk.')
            print('    Decrease: velocity.')
        
        # generate trajectory
        param = ruckig.InputParameter(1)
        param.max_velocity = [vmax]
        param.max_acceleration = [amax]
        param.max_jerk = [jmax]
        if not self._worst_case_checkbox.value:
            param.target_position = [s]
        else:
            _, vc, ac = jerk_limit_profile(m, vmax, amax, Um, state=True)
            param.current_position = [0.0]
            param.current_velocity = [vmax - amax**2/jmax]
            param.target_velocity = [vmax]
            param.target_position = [2*vmax*amax/jmax - amax**3/jmax**2]
        ruck = ruckig.Ruckig(1)
        traj = ruckig.Trajectory(1)
        ruck.calculate(param, traj)
        t = np.arange(0.0, traj.duration, T)
        q = np.vectorize(lambda t: np.array(traj.at_time(t)).flatten(), signature='()->(n)')(t).T
        j = np.diff(q[2,:], append=q[2,-1])/T

        # electrical paramters
        Iq = (m.J*q[2,:]) / (3/2*m.N*m.Fm)
        dIq = (m.J*j) / (3/2*m.N*m.Fm)
        Uds = - q[1,:]*m.N*m.Lq*Iq
        Uqs = m.R*Iq + q[1,:]*m.N*m.Fm 
        Ud = Uds
        Uq = Uqs + m.L*dIq
        maxIq = np.max(np.abs(Iq))
        if maxIq <= Im:
            cprint(f'Current limit is not reached.', color='green')
            print(f'    Max current {maxIq:.2f}.')
            print(f'    Current limit {Im:.2f}.')
        else:
            cprint(f'Current limit IS REACHED: max current {maxIq}.', color='red')
            print(f'    Max current {maxIq:.2f}.')
            print(f'    Current limit {Im:.2f}.')
        maxUs = np.max(np.sqrt(Ud**2 + Uq**2))
        if maxUs <= Um:
            cprint(f'Voltage limit is not reached.', color='green')
            print(f'    Max voltage {maxUs:.2f}.')
            print(f'    Voltage limit {Um:.2f}.')
        else:
            cprint(f'Voltage limit IS REACHED.', color='red')
            print(f'    Max voltage {maxUs:.2f}.')
            print(f'    Voltage limit {Um:.2f}.')

        # plot trajectory
        display(HTML('<h3>Trajectory plots</h3>'))
        v = np.array([1/T1, 1/T2, 1/T3])
        fig, axes = plt.subplots(4,2, figsize = (15, 15))
        for name, k, units in zip(['postion','speed','accel'], range(3), ['rad','rad/s','rad/s^2']):
            # plot trajectory
            ax = axes[k, 0]
            ax.plot(t, q[k,:], label=name)
            ax.set_xlabel('t, s')
            ax.set_ylabel(f'{name}, {units}')
            ax.set_title(name)
            ax.grid(True)
            # plot spectrum
            ax = axes[k, 1]
            # fft of signal
            N = 2*q.shape[1]
            s = np.concatenate((q[k,:], np.flip(q[k,:])))
            S = T*np.fft.fft(s) / np.std(s)
            w = np.linspace(0.0, (1/T), N, endpoint=False)
            ax.loglog(w[1:int(N/2)], np.abs(S[1:int(N/2)]), label='numeric')
            # theoretical
            N *= 8
            w = np.linspace(0.0, (1/T), N, endpoint=False)
            S = np.ones(w.shape)
            for l in range(3):
                if l >= k: 
                    S *= np.abs(np.sinc(w/v[l]))
            else:
                #S *= np.abs(2.0/v[l]*np.sin(0.5*w/v[l]))
                pass
            ax.loglog(w[1:int(N/2)], np.abs(S[1:int(N/2)]), label='theretical')
            ax.set_title(name + ' spectrum')
            ax.set_ylim(1e-4, 10)
            ax.grid(True)
            ax.legend()
        # plot current
        ax = axes[3,0]
        ax.plot(t, Iq, label='Iq')
        ax.set_xlabel('t, s')
        ax.set_ylabel('I, A')
        ax.set_title('Current')
        ax.grid(True)
        ax.legend()
        # plot voltage
        ax = axes[3,1]
        ax.plot(t, Uqs, label='Uq static')
        ax.plot(t, Uds, label='Ud static')
        ax.plot(t, Uq, label='Uq')
        ax.plot(t, Ud, label='Ud')
        ax.set_xlabel('t, s')
        ax.set_ylabel('U, V')
        ax.set_title('Voltages')
        ax.grid(True)
        ax.legend()   
        
        # display plot
        plt.show()

form = ProfileForm(model_form.model)

### Варианты лимита рывка

In [ ]:
N = 100
m = model_form.model
Imax, Umax = form.limit_params()
_, vmax, amax, _ = form.profile_params()

# jerk grid
v = np.linspace(0.0, vmax, N)
a = np.linspace(0.0, amax, N)
v, a = np.meshgrid(v, a)
# calculate jerk limit
jmax_abs = np.vectorize(lambda v, a: jerk_limit(m, v, a, Umax))(v, a)
jmax_profile = np.vectorize(lambda v, a: jerk_limit_profile(m, v, a, Umax))(v, a)
# plot
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')
# Plot a basic wireframe.
gcount = 30
ax.plot_wireframe(v, a, jmax_abs, rcount=gcount, ccount=gcount, label = 'jmax absolute', color='red')
ax.plot_wireframe(v, a, jmax_profile, rcount=gcount, ccount=gcount, label = 'jmax profile', color='blue')
ax.legend()
ax.grid(True)
plt.show()


### Моделирование режима MTPA в неподвижной СК

In [ ]:
# max volatge and current
Um = 120
Im = 8

# current regulator
t_Ireg = 0.0006
k = 3/t_Ireg
ireg = PIParameters(m.L*k, m.R*k)

# plot paramters
n_points = 1000
vlim_rpm = np.array([1, 2000])
Tlim = np.array([0, m.Tn*5])
vlim = rpm_to_rads(vlim_rpm)
v = np.linspace(vlim[0], vlim[1], n_points)

Id = np.ndarray(shape=v.shape)
Iq = np.ndarray(shape=v.shape)
Ud = np.ndarray(shape=v.shape)
Uq = np.ndarray(shape=v.shape)

for k in range(len(v)):
    _, (Id[k], Iq[k]), (Ud[k], Uq[k]) = MTPA_in_ABframe(m, v[k], Im = Im, Um = Um, ireg = ireg)

fig, (ax1, ax2) = plt.subplots(2,1)

ax1.plot(rads_to_rpm(v), Id, label='Id')
ax1.plot(rads_to_rpm(v), Iq, label='Iq')
ax1.plot(rads_to_rpm(v), np.sqrt(Id**2 + Iq**2), label='|Idq|')
ax1.set_xlabel('v, rmp')
ax1.set_ylabel('I, A')
ax1.legend()
ax1.grid(True)

ax2.plot(rads_to_rpm(v), Ud, label='Ud')
ax2.plot(rads_to_rpm(v), Uq, label='Uq')
ax2.plot(rads_to_rpm(v), np.sqrt(Ud**2 + Uq**2), label='|Udq|')
ax2.set_xlabel('v, rmp')
ax2.set_ylabel('U, V')
ax2.legend()
ax2.grid(True)

plt.show()